In [1]:
from BanditAnalysisFunctions import *
%matplotlib qt
pd.options.display.max_rows = 500
#before
df1 = pd.read_csv('sessdf1.csv')
df2 = pd.read_csv('sessdf2.csv')
df3 = pd.read_csv('sessdf3.csv')
df4 = pd.read_csv('sessdf4.csv')
#after
df1_new = pd.read_csv('sessdf1_new.csv')
df2_new = pd.read_csv('sessdf2_new.csv')
df3_new = pd.read_csv('sessdf3_new.csv')
df4_new = pd.read_csv('sessdf4_new.csv')

In [169]:
def switchProb_trial(df, sess_len, include_sess, subset = True, animal = '', start = 'all'):
    '''
    return switch probability as a function of trial number
    inputs:
    sessdf: original animal df
    sess_len: number of initial trials to be considered
    include_sess: last n sessions to be considered
    subset: bool, whether to subset str sessions from unstr dataframes
    animal: name of animal 
    start: whether the animal started the session with a correct or incorrect choice ('correct' /'incorrect'/'all'), default = 'all'
    outputs:
    array of switch probs averaged across sessions
    '''
    print('no. of sess before subsetting: ', len(pd.unique(df['session#'])))
    
    if subset == True:
        df = df[((df['rewprobfull1']==10)&(df['rewprobfull2']==90))|((df['rewprobfull1']==20)&(df['rewprobfull2']==80))|((df['rewprobfull1']==30)&(df['rewprobfull2']==70))|((df['rewprobfull1']==40)&(df['rewprobfull2']==60|((df['rewprobfull1']==90)&(df['rewprobfull2']==10))|((df['rewprobfull1']==80)&(df['rewprobfull2']==20))|((df['rewprobfull1']==70)&(df['rewprobfull2']==30))|((df['rewprobfull1']==60)&(df['rewprobfull2']==40))))]    
    # print(df.groupby('session#').mean(numeric_only = True))
    df = df[['trial#', 'session#','port', 'reward']] 
    sessions = pd.unique(df['session#'])
    print('No. of sess after subsetting: ', len(sessions))
    # print(sessions)
    # print(np.max(sessions))
    exclude = np.max(sessions)- include_sess ##### this assumes no subsetting
    # print('ex: ', exclude)
    # print('dfb: ', df)
    # df = df[df['session#']>exclude] #only keep n sessions specified in include_sess
    # print('dfa: ', df)
    switches = abs(np.diff(df['port'].tolist()))
    switches = np.insert(switches, 0, 0, axis = 0) #convert choices to 0/1:stay/switch encoding and add a 0 to the first row
    df.insert(2, 'switch', switches ) 

    df = df.groupby('session#').filter(lambda x: len(x['trial#'])>sess_len) # filter sessions to include only those longer than sess_len
    df = df.groupby('session#').head(sess_len)
    sessions_sub = pd.unique(df['session#'])
    array_switchProbs = []
    for i in range(len(sessions_sub)):
        session = sessions_sub[i]   
        print('session: ', session)
        switchProbs = []
        trials = 2
        if start == 'correct': 
            df_sub = df[(df['session#']==session)&df['reward'][0] == 1]
        elif start == 'incorrect':
            df_sub = df[(df['session#']==session)&df['reward'][0] == 0]
        elif start == 'all':
            df_sub = df[(df['session#']==session)]
            
        print(df_sub.head(2))
       
        # for e in range(sess_len):
        #     sum_switches = np.sum((df_sub['switch'].head(trials)).tolist())
        #     switch_prob = sum_switches/trials
        #     switch_prob = 
        #     switchProbs.append(switch_prob)
        #     trials += 1

        switchProbs = df_sub['switch'].rolling(25).mean(numeric_only= True)
        
        array_switchProbs.append(switchProbs)
    av_switchProbs = np.mean(array_switchProbs, axis = 0)
    fig, ax = plt.subplots(layout = 'tight')
    ax.plot(range(sess_len), av_switchProbs)
    ax.set_xlabel('trial#')
    ax.set_ylabel('switch probability')
    ax.set_ylim(0, 0.1)
    plt.title(f'{animal}')
    sns.despine()
    plt.show()
    return
            

In [176]:
switchProb_trial(df4_new, 250, 0,  animal = 'Grump_str', start = 'all', subset = True)

no. of sess before subsetting:  546
No. of sess after subsetting:  184
session:  2
     trial#  session#  switch  port  reward
225     225         2     0.0   1.0       0
226     226         2     0.0   1.0       0
session:  6
      trial#  session#  switch  port  reward
1347    1347         6     1.0   1.0       0
1348    1348         6     0.0   1.0       0
session:  10
      trial#  session#  switch  port  reward
2578    2578        10     1.0   1.0       0
2579    2579        10     0.0   1.0       0
session:  13
      trial#  session#  switch  port  reward
3198    3198        13     0.0   2.0       1
3199    3199        13     0.0   2.0       1
session:  16
      trial#  session#  switch  port  reward
3936    3936        16     0.0   2.0       1
3937    3937        16     0.0   2.0       1
session:  29
      trial#  session#  switch  port  reward
7017    7017        29     0.0   2.0       1
7018    7018        29     0.0   2.0       1
session:  49
       trial#  session#  switch  

Do rats carry their choice over to subsequent sessions?


In [45]:
df = filter_by_time(df3, t_diff = 0)

In [46]:
last = df.groupby('session#').tail(1)['port'].reset_index(drop = True)


In [47]:
first = df.groupby('session#').head(1)['port'].reset_index(drop = True).shift(-1)


In [48]:
choices = pd.DataFrame(data = [last.values, first.values]).transpose()
choices = choices.rename(columns = {0:'last choice', 1:'first chocie'})
# choices

In [49]:
compare = choices.where(choices['last choice']==choices['first chocie'], 0)['last choice']
compare = compare.replace({2:1})
# print(compare)
np.mean(compare)

0.7753623188405797

In [34]:
df.groupby('session#').mean()

C:\Users\dlab\AppData\Local\Temp\ipykernel_11524\3007122106.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('session#').mean()


,Unnamed: 0,trial#,trialstart,port,reward,trialend,eptime,task,rewprobfull1,rewprobfull2,rw,seconds
session#,,,,,,,,,,,,
1,151.0,151.0,4.455764e+05,1.983498,0.663366,4.461450e+05,1.695838e+09,12.0,30.0,70.0,69.339934,64745.069307
4,657.5,657.5,3.518695e+05,1.968750,0.897321,3.524403e+05,1.695900e+09,12.0,10.0,90.0,87.500000,41110.013393
5,895.5,895.5,4.288364e+05,1.980159,0.797619,4.294761e+05,1.695901e+09,12.0,20.0,80.0,78.809524,41783.432540
6,1355.0,1355.0,9.111933e+05,1.989505,0.301349,9.117837e+05,1.695902e+09,12.0,70.0,30.0,30.419790,43123.833583
7,1765.0,1765.0,1.432518e+05,1.934641,0.418301,1.437960e+05,1.695903e+09,12.0,60.0,40.0,41.307190,44188.581699
8,1986.5,1986.5,5.330921e+05,1.975862,0.693103,5.336944e+05,1.695920e+09,12.0,30.0,70.0,69.034483,60971.158621
10,2471.0,2471.0,1.166732e+05,1.933333,0.186667,1.172905e+05,1.695923e+09,12.0,80.0,20.0,24.000000,63563.000000
11,2669.5,2669.5,4.394994e+05,1.922360,0.624224,4.400597e+05,1.695986e+09,12.0,40.0,60.0,58.447205,40591.118012
12,2947.0,2947.0,2.625008e+05,1.991416,0.862661,2.630428e+05,1.695987e+09,12.0,10.0,90.0,89.313305,41195.064378


In [39]:
df[df['session#']==125]

,Unnamed: 0,trial#,trialstart,port,reward,trialend,session#,eptime,task,rewprobfull1,rewprobfull2,rw,animal,datetime,seconds
73244,73244,73244,86071,1,0,86571,125,1700089297,12,10,90,10,Gronckle,2023-11-15 23:01:37,82897
73245,73245,73245,87912,1,0,88412,125,1700089299,12,10,90,10,Gronckle,2023-11-15 23:01:39,82899
73246,73246,73246,89083,1,0,89656,125,1700089300,12,10,90,10,Gronckle,2023-11-15 23:01:40,82900
73247,73247,73247,92079,1,0,92579,125,1700089303,12,10,90,10,Gronckle,2023-11-15 23:01:43,82903
73248,73248,73248,93209,1,0,93709,125,1700089304,12,10,90,10,Gronckle,2023-11-15 23:01:44,82904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73992,73992,73992,2343183,2,1,2343683,125,1700091554,12,10,90,90,Gronckle,2023-11-15 23:39:14,85154
73993,73993,73993,2378394,2,0,2383895,125,1700091589,12,10,90,90,Gronckle,2023-11-15 23:39:49,85189
73994,73994,73994,2384680,2,1,2385180,125,1700091595,12,10,90,90,Gronckle,2023-11-15 23:39:55,85195
73995,73995,73995,2386260,2,1,2386817,125,1700091597,12,10,90,90,Gronckle,2023-11-15 23:39:57,85197


In [63]:
57+64+59+56+39+70+86+46

477

In [64]:
477/8

59.625